In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/r-street-quant-challenge/sample_submission.csv
/kaggle/input/r-street-quant-challenge/train_target.csv
/kaggle/input/r-street-quant-challenge/train_data.csv
/kaggle/input/r-street-quant-challenge/test_data.csv


In [2]:
import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv('../input/r-street-quant-challenge/train_data.csv')
target = pd.read_csv('../input/r-street-quant-challenge/train_target.csv')
test = pd.read_csv('../input/r-street-quant-challenge/test_data.csv')

In [4]:
# train['target'] = target['target']

In [5]:
train['date'] = pd.to_datetime(train['date'])
# train['year'] = train['date'].apply(lambda x: x.year)
# train['month'] = train['date'].apply(lambda x: x.month)
# train['hour'] = train['date'].apply(lambda x: x.time().hour) #
# train['minute'] = train['date'].apply(lambda x: x.time().minute)
train['hour+min'] = ((train['date'].apply(lambda x: x.time().hour))*60 + train['date'].apply(lambda x: x.time().minute))/1000
# train['weekday'] = train['date'].apply(lambda x: x.weekday())

test['date'] = pd.to_datetime(test['date'])
# test['year'] = test['date'].apply(lambda x: x.year)
# test['month'] = test['date'].apply(lambda x: x.month)
# test['hour'] = test['date'].apply(lambda x: x.time().hour)
# test['minute'] = test['date'].apply(lambda x: x.time().minute)
test['hour+min'] = ((test['date'].apply(lambda x: x.time().hour))*60 + test['date'].apply(lambda x: x.time().minute))/1000
# test['weekday'] = test['date'].apply(lambda x: x.weekday())

In [6]:
train['close_higher_than_open'] = train['open'] < train['close']
train['is RSI 7 greater than 21'] = train['RSI_7'] > train['RSI_21']
train['is SMA 7 greater than 21'] = train['SMA_7'] > train['SMA_21']

train['percent_jump_from_low_to_high'] = (train['high']-train['low'])/train['low']
train['close_higher_than_open'] = train['close_higher_than_open'].apply(lambda x: 1 if x==True else 0)
train['is RSI 7 greater than 21'] = train['is RSI 7 greater than 21'].apply(lambda x: 1 if x==True else 0)
train['is SMA 7 greater than 21'] = train['is SMA 7 greater than 21'].apply(lambda x: 1 if x==True else 0)

In [7]:
test['close_higher_than_open'] = test['open'] < test['close']
test['is RSI 7 greater than 21'] = test['RSI_7'] > test['RSI_21']
test['is SMA 7 greater than 21'] = test['SMA_7'] > test['SMA_21']

test['percent_jump_from_low_to_high'] = (test['high']-test['low'])/test['low']
test['close_higher_than_open'] = test['close_higher_than_open'].apply(lambda x: 1 if x==True else 0)
test['is RSI 7 greater than 21'] = test['is RSI 7 greater than 21'].apply(lambda x: 1 if x==True else 0)
test['is SMA 7 greater than 21'] = test['is SMA 7 greater than 21'].apply(lambda x: 1 if x==True else 0)

In [8]:
overbought = [0 for i in range(train.shape[0])]
oversold = [0 for i in range(train.shape[0])]
for i in range(train.shape[0]):
    s = 0
    if train['RSI_7'][i]>70:
        s += 1
#     if train['RSI_14'][i]>70:
#         s += 1
#     if train['RSI_21'][i]>70:
#         s += 1
    overbought[i] = s

for i in range(train.shape[0]):
    s = 0
    if train['RSI_7'][i]<30:
        s += 1
#     if train['RSI_14'][i]<30:
#         s += 1
#     if train['RSI_21'][i]<30:
#         s += 1
    oversold[i] = s
    
train['Overbought'] = overbought
train['Oversold'] = oversold

In [9]:
overbought = [0 for i in range(test.shape[0])]
oversold = [0 for i in range(test.shape[0])]
for i in range(test.shape[0]):
    s = 0
    if test['RSI_7'][i]>70:
        s += 1
#     if test['RSI_14'][i]>70:
#         s += 1
#     if test['RSI_21'][i]>70:
#         s += 1
    overbought[i] = s

for i in range(test.shape[0]):
    s = 0
    if test['RSI_7'][i]<30:
        s += 1
#     if test['RSI_14'][i]<30:
#         s += 1
#     if test['RSI_21'][i]<30:
#         s += 1
    oversold[i] = s
    
test['Overbought'] = overbought
test['Oversold'] = oversold

In [10]:
# rsis = ['RSI_7', 'RSI_14', 'RSI_21']
# for rsi in rsis:
#     train[rsi] = train[rsi].apply(lambda x: (x-30)/40)
#     test[rsi] = test[rsi].apply(lambda x: (x-30)/40)

In [11]:
# trainv2 = train.append(test)
# trainv2

In [12]:
# I fear normalizing data will leak the data but i want the numbers to be on the same scale so i will find percent change for each row from a timestamp before
normalizing_cols = ['open', 'high', 'low', 'close','SMA_7','SMA_14', 'SMA_21', 'RSI_7', 'RSI_14', 'RSI_21', 'ATR_14', 'bb_mavg','bb_hband', 'bb_lband']
# 
for col in normalizing_cols:
#     train[col + '_old'] = train[col].tolist()
#     test[col + '_old'] = test[col].tolist()
    
    train[col] = train[col].pct_change()
    train[col][0] = 0
    test[col] = test[col].pct_change()
    test[col][0] = 0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [13]:
train.drop(index=[0],inplace=True)
target.drop(index=[0],inplace=True)

In [14]:
# No use of date, day, week rn -> (can't find a way to use these rn)
# train.drop(columns = ['Id','date','day','week'],inplace=True)
train.drop(columns = ['Id','date'],inplace=True)
train['day'] = train['day']/31
train['week'] = train['week']/54
# test.drop(columns = ['Id','date','day','week'],inplace=True)
test.drop(columns = ['Id','date'],inplace=True)
test['day'] = test['day']/31
test['week'] = test['week']/54
target.drop(columns = ['Id'],inplace=True)

In [15]:
# train.drop(columns = ['year'],inplace=True)

In [16]:
# useless_feature = ['week','percent_jump_from_low_to_high','RSI_7']
# train.drop(columns=useless_feature,inplace=True)
# test.drop(columns=useless_feature,inplace=True)

In [17]:
x_train = train.iloc[:int(0.8*train.shape[0])]
x_test = train.iloc[int(0.8*train.shape[0]):] 
y_train = target.iloc[:int(0.8*target.shape[0])]
y_test = target.iloc[int(0.8*target.shape[0]):]

In [18]:
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LinearRegression 

In [19]:
# ligr = LinearRegression()
# ligr.fit(x_train,y_train)
# prediction = ligr.predict(x_test)
# roc_auc_score(y_test,prediction)

In [20]:
log = LogisticRegression(class_weight='balanced')

In [21]:
# from sklearn.feature_selection import RFE
# predictor = x_train
# selector = RFE(log,n_features_to_select=1)
# selector.fit(predictor,y_train)
# order = selector.ranking_
# feature_ranks = []
# for i in order:
#     feature_ranks.append(f"{i}.{x_train.columns[i-1]}")
# #     print(x_train.columns[i-1])
    

In [22]:
# feature_ranks
# x_train.columns

In [23]:
from sklearn.feature_selection import SelectFromModel
smf = SelectFromModel(log,threshold=-np.inf,max_features=21)
smf.fit(x_train,y_train)
feature_idx = smf.get_support()
feature_name = x_train.columns[feature_idx]
feature_name

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Index(['open', 'high', 'low', 'close', 'day', 'SMA_7', 'SMA_14', 'SMA_21',
       'RSI_14', 'RSI_21', 'ATR_14', 'bb_mavg', 'bb_hband', 'bb_lband',
       'hour+min', 'close_higher_than_open', 'is RSI 7 greater than 21',
       'is SMA 7 greater than 21', 'percent_jump_from_low_to_high',
       'Overbought', 'Oversold'],
      dtype='object')

In [24]:
x_train_imp = x_train[feature_name]
x_test_imp = x_test[feature_name]

In [25]:

log.fit(x_train_imp,y_train)
prediction = log.predict_proba(x_test_imp)[:,1]

# log.fit(x_train,y_train)
# prediction = log.predict(x_test)
roc_auc_score(y_test,prediction)
## 0.516706719215611

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.5139817360376814

In [26]:
log.classes_

array([0, 1])

In [27]:
# 21- 0.5174872632894357

In [28]:
from sklearn.ensemble import RandomForestClassifier
rfg = RandomForestClassifier(class_weight='balanced')
rfg.fit(x_train,y_train)
prediction = rfg.predict(x_test)
roc_auc_score(y_test,prediction)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.5032798231279438

In [29]:
# from sklearn.feature_selection import mutual_info_classif
# importances = mutual_info_classif(x_train,y_train)
# feat_importances = pd.Series(importances,x_train.columns)
# plt.figure(figsize=(10,10))
# feat_importances.plot(kind='barh',color = 'teal')
# plt.show()

In [30]:
from xgboost import XGBClassifier 
xgbc = XGBClassifier()
xgbc.fit(x_train,y_train)
prediction = xgbc.predict(x_test)
roc_auc_score(y_test,prediction)
# best score -> 0.5157666057867922 by taking all those time features and everything

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[09:52:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.49956743247140245

In [31]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(loss_function='CrossEntropy') 
clf.fit(x_train,y_train)

# # 0.5134288186100163

0:	learn: 0.6930046	total: 65.8ms	remaining: 1m 5s
1:	learn: 0.6928558	total: 73.5ms	remaining: 36.7s
2:	learn: 0.6926664	total: 81.8ms	remaining: 27.2s
3:	learn: 0.6925084	total: 89.5ms	remaining: 22.3s
4:	learn: 0.6923399	total: 98.1ms	remaining: 19.5s
5:	learn: 0.6921914	total: 106ms	remaining: 17.6s
6:	learn: 0.6920319	total: 114ms	remaining: 16.2s
7:	learn: 0.6918770	total: 122ms	remaining: 15.1s
8:	learn: 0.6917064	total: 130ms	remaining: 14.3s
9:	learn: 0.6915614	total: 137ms	remaining: 13.6s
10:	learn: 0.6914134	total: 146ms	remaining: 13.1s
11:	learn: 0.6912694	total: 154ms	remaining: 12.7s
12:	learn: 0.6911330	total: 163ms	remaining: 12.3s
13:	learn: 0.6909689	total: 170ms	remaining: 12s
14:	learn: 0.6908552	total: 179ms	remaining: 11.7s
15:	learn: 0.6907332	total: 187ms	remaining: 11.5s
16:	learn: 0.6906323	total: 195ms	remaining: 11.3s
17:	learn: 0.6905021	total: 203ms	remaining: 11.1s
18:	learn: 0.6903486	total: 210ms	remaining: 10.9s
19:	learn: 0.6901850	total: 218ms	rema

In [32]:
prediction = clf.predict(x_test)
roc_auc_score(y_test,prediction)

0.507330577717966

In [33]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(x_train, y_train)
prediction = classifier.predict(x_test)
roc_auc_score(y_test,prediction)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.5060847832356051

In [34]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(class_weight = "balanced")
sgd.fit(x_train,y_train)
prediction = sgd.predict(x_test)
roc_auc_score(y_test,prediction)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.5097914063250986

In [35]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(x_train,y_train)
prediction = knn.predict(x_test)
roc_auc_score(y_test,prediction)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.4982408920503701

In [36]:
from sklearn.svm import SVC
svm = SVC(kernel = "linear", class_weight= "balanced",probability=True)
svm.fit(x_train,y_train)
prediction = svm.predict_proba(x_test)[:,1]
roc_auc_score(y_test,prediction)
# 0.5171796597135442

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.5226369316543304

## Making Prediction

In [37]:
ligr = LinearRegression()
ligr.fit(train,target)
prediction = ligr.predict(test)

In [38]:
trainv2 = train[feature_name]
testv2 = test[feature_name]

In [39]:
log = LogisticRegression(class_weight='balanced')
# log.fit(train,target)
# prediction = log.predict(test)

log.fit(trainv2,target)
prediction = log.predict(testv2)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [40]:
# clf = CatBoostClassifier(loss_function='CrossEntropy') 
# clf.fit(train,target)
# prediction = clf.predict(test)

In [41]:
from sklearn.svm import SVC
svm = SVC(kernel = "linear", class_weight= "balanced",probability=True)
svm.fit(train,target)
prediction = svm.predict_proba(test)[:,1]

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [42]:
sub = pd.read_csv('../input/r-street-quant-challenge/sample_submission.csv')
sub['target'] = prediction
sub

,Id,target
0,0,0.523538
1,1,0.523539
2,2,0.523538
3,3,0.494559
4,4,0.494560
...,...,...
10312,10312,0.494557
10313,10313,0.523534
10314,10314,0.523536
10315,10315,0.523534


In [43]:
sub.to_csv('SVC_Predict_proba_submission.csv',index=False)

In [44]:
prediction

array([0.52353817, 0.52353898, 0.52353846, ..., 0.52353571, 0.52353397,
       0.52353454])

In [45]:
sub.head(60)

,Id,target
0,0,0.523538
1,1,0.523539
2,2,0.523538
3,3,0.494559
4,4,0.494560
5,5,0.494558
6,6,0.494560
7,7,0.494562
8,8,0.494562
9,9,0.494562
